In [1]:
from datetime import datetime
import json
import os
import re
from pathlib import Path
from typing import Dict, List
from urllib.parse import parse_qs
from bs4 import BeautifulSoup, Tag
import scrapy
from scrapy import signals
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from scrapy.http import HtmlResponse
from scrapy_playwright.page import PageMethod
from playwright.sync_api import sync_playwright, Playwright
import requests
import asyncio
from functional import seq
from typing import Optional
from playwright.async_api import async_playwright, Playwright


In [16]:
link= 'https://www.kcl.ac.uk/study/postgraduate-taught/courses/psychology-and-neuroscience-of-mental-health-mscpgdippgcert'
response= requests.get(link)
soup= BeautifulSoup(response.content, 'html.parser')

In [23]:
text= "MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years."
if "." in text:
    values= text.split('.')
    for value in values:
        if 'MSc' in value:
            duration= value
            print(duration)
text="One year full-time or two years part-time"
if 'or' in text:
    values= text.split('or')
    for value in values:
        if 'part-time' in value:
            duration= value
            study_mode= 'part-time'
        elif 'full-time' in value:
            duration= value
            study_mode= 'full-time'
        print(duration, study_mode)
text= "1 year FT/2 years PT"
if '/' in text:
    values= text.split('/')
    for value in values:
        if 'PT' in value:
            duration= value
            study_mode= 'part-time'
        elif 'FT' in value:
            duration= value
            study_mode= 'full-time'
        print(duration, study_mode)


MSc: Minimum two years, maximum six years
One year full-time  full-time
 two years part-time part-time
1 year FT full-time
2 years PT part-time


In [12]:
def _get_entry_requirements(json_data: dict) -> Optional[str]:
    try:
        try:
            basic_requirements = json_data['routing']['entry']['entryRequirement']['pgtDefault']
            other_requirements = json_data['routing']['entry']['otherRequirements']

            entry_requirements = "".join(seq([basic_requirements, other_requirements]).filter(lambda x: x).to_list())
        except:
            entry_requirements = json_data['routing']['entry']['entryRequirement']
        try:
            entry_requirements= json_data['routing']['entry']['otherRequirements']
        except:
            pass
    
    except (AttributeError, TypeError):
        entry_requirements = ""
    return entry_requirements
entry_requirements= _get_entry_requirements(data)
print(entry_requirements)

<p>The normal entry requirement for the BSc is a Diploma in Higher Education (120 Level 5 credits) (or overseas equivalent). Applicants must be registered nurses with the NMC or an equivalent governing body for international applicants. IELTS 6.5 overall, minimum of 6.0 in all skills.</p>
<p>Applicants must be qualified nurses with a current registration either in the UK or home country. Applicants will have a diploma in nursing that has been awarded by a recognised academic institution.</p>
<p>Visit our admissions webpages to view our <a href="https://www.kcl.ac.uk/study/postgraduate-taught/how-to-apply/entry-requirements/english-language-requirements">English language entry requirements</a></p>


In [15]:
import requests

cookies = {
    'OptanonAlertBoxClosed': '2024-03-28T08:51:00.450Z',
    '_gid': 'GA1.3.509480863.1711833552',
    '_fbp': 'fb.2.1711902939866.201974845',
    '__hs_cookie_cat_pref': '1:true_2:true_3:true',
    'hubspotutk': '338f7eef2923d6365b019b9516e5b2ec',
    '__hssrc': '1',
    '_gcl_au': '1.1.1626541155.1711902948',
    '_hjSessionUser_3882840': 'eyJpZCI6IjNjZDAxM2Y3LWU1YjEtNTBlZC1hZjUzLWI4YTFmODJiMDk3NiIsImNyZWF0ZWQiOjE3MTE5MDI5NDExNzYsImV4aXN0aW5nIjp0cnVlfQ==',
    '__hstc': '102787167.338f7eef2923d6365b019b9516e5b2ec.1711902945790.1711914310426.1711924218847.3',
    '_ga_2B2LNW4G7M': 'GS1.1.1711947718.6.0.1711947718.0.0.0',
    '_ga_MB7LPD0MRQ': 'GS1.1.1711950710.7.1.1711952107.0.0.0',
    '_ga': 'GA1.3.1650481729.1711615941',
    '_gat_gtag_UA_228896_1': '1',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Mon+Apr+01+2024+13%3A51%3A53+GMT%2B0600+(Bangladesh+Standard+Time)&version=202402.1.0&browserGpcFlag=0&isIABGlobal=false&consentId=1aec5d0b-87c7-45e8-8e05-b61c16202a81&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&hosts=H105%3A1%2CH148%3A1%2CH103%3A1%2CH130%3A1%2CH24%3A1%2CH104%3A1%2CH1%3A1%2CH201%3A1%2CH179%3A1%2CH19%3A1%2CH20%3A1%2CH144%3A1%2CH181%3A1%2CH117%3A1%2CH33%3A1%2CH48%3A1%2CH195%3A1%2CH9%3A1%2CH11%3A1%2CH106%3A1%2CH204%3A1%2CH57%3A1%2CH58%3A1%2CH59%3A1%2CH202%3A1%2CH21%3A1%2CH114%3A1%2CH115%3A1%2CH116%3A1%2CH23%3A1%2CH149%3A1%2CH173%3A1%2CH203%3A1%2CH150%3A1%2CH185%3A1%2CH126%3A1%2CH161%3A1%2CH127%3A1%2CH156%3A1%2CH128%3A1%2CH140%3A1%2CH198%3A1%2CH180%3A1%2CH62%3A1%2CH200%3A1%2CH187%3A1%2CH25%3A1%2CH26%3A1&genVendors=&geolocation=BD%3BC&AwaitingReconsent=false',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'accesstoken': 'JjV9NgvYm8BQgTNx2AtThsRBeK5qZxArDnRc2SKrzYWzvsS6',
    # 'cookie': 'OptanonAlertBoxClosed=2024-03-28T08:51:00.450Z; _gid=GA1.3.509480863.1711833552; _fbp=fb.2.1711902939866.201974845; __hs_cookie_cat_pref=1:true_2:true_3:true; hubspotutk=338f7eef2923d6365b019b9516e5b2ec; __hssrc=1; _gcl_au=1.1.1626541155.1711902948; _hjSessionUser_3882840=eyJpZCI6IjNjZDAxM2Y3LWU1YjEtNTBlZC1hZjUzLWI4YTFmODJiMDk3NiIsImNyZWF0ZWQiOjE3MTE5MDI5NDExNzYsImV4aXN0aW5nIjp0cnVlfQ==; __hstc=102787167.338f7eef2923d6365b019b9516e5b2ec.1711902945790.1711914310426.1711924218847.3; _ga_2B2LNW4G7M=GS1.1.1711947718.6.0.1711947718.0.0.0; _ga_MB7LPD0MRQ=GS1.1.1711950710.7.1.1711952107.0.0.0; _ga=GA1.3.1650481729.1711615941; _gat_gtag_UA_228896_1=1; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Apr+01+2024+13%3A51%3A53+GMT%2B0600+(Bangladesh+Standard+Time)&version=202402.1.0&browserGpcFlag=0&isIABGlobal=false&consentId=1aec5d0b-87c7-45e8-8e05-b61c16202a81&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&hosts=H105%3A1%2CH148%3A1%2CH103%3A1%2CH130%3A1%2CH24%3A1%2CH104%3A1%2CH1%3A1%2CH201%3A1%2CH179%3A1%2CH19%3A1%2CH20%3A1%2CH144%3A1%2CH181%3A1%2CH117%3A1%2CH33%3A1%2CH48%3A1%2CH195%3A1%2CH9%3A1%2CH11%3A1%2CH106%3A1%2CH204%3A1%2CH57%3A1%2CH58%3A1%2CH59%3A1%2CH202%3A1%2CH21%3A1%2CH114%3A1%2CH115%3A1%2CH116%3A1%2CH23%3A1%2CH149%3A1%2CH173%3A1%2CH203%3A1%2CH150%3A1%2CH185%3A1%2CH126%3A1%2CH161%3A1%2CH127%3A1%2CH156%3A1%2CH128%3A1%2CH140%3A1%2CH198%3A1%2CH180%3A1%2CH62%3A1%2CH200%3A1%2CH187%3A1%2CH25%3A1%2CH26%3A1&genVendors=&geolocation=BD%3BC&AwaitingReconsent=false',
    'dnt': '1',
    'referer': 'https://www.kcl.ac.uk/study/postgraduate-taught/courses/applied-neuroscience-msc',
    'sec-ch-ua': '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
}

params = {
    'linkDepth': '1',
}

response = requests.get(
    'https://www.kcl.ac.uk/api/delivery/projects/website/entries/72e8ecb1-123e-4621-bfef-0cbf249eb40e',
    params=params,
    cookies=cookies,
    headers=headers,
)
data2=response.json()
with open('data2.json', 'w') as f:
    json.dump(data2, f, indent=4)

In [108]:
try:
    mapper = {
        'ukFee': ('uk', 'Full-time'),
        'internationalFee': ('international', 'Full-time'),
        'ukPartTimeFee': ('uk', 'Part-time'),
        'internationalPartTimeFee': ('international', 'Part-Time')
    }

    # # TODO: extract durations
    tuitions = []
    try:
        for k in seq(mapper.keys()).filter(lambda x: x in json_data['routing']['entry']):
            fee = json_data['routing']['entry'][k]
            duration = json_data['routing']['entry']['duration']
            if not fee:
                continue

            fee = re.findall(r'£\d+,\d+', fee).pop(0)

            student_category, study_mode = mapper[k]
            tuitions.append({
                'study_mode': study_mode,
                'duration': duration,
                'student_category': student_category,
                'fee': fee
            })
    except:
        pass
    if(len(tuitions) == 0):
        try:
            for k in seq(mapper.keys()).filter(lambda x: x in json_data['routing']['entry']['relatedCourses'][0]['relatedCourses'][0]):
                fee=json_data['routing'] ['entry'] ['relatedCourses'][0]['relatedCourses'][0][k]
                duration= json_data['routing']['entry']['relatedCourses'][0]['relatedCourses'][0]['duration']

                if not fee:
                    continue

                fee = re.findall(r'£\d+,\d+', fee).pop(0)
        
                student_category, study_mode = mapper[k]
                tuitions.append({
                    'study_mode': study_mode,
                    'duration': duration,
                    'student_category': student_category,
                    'fee': fee
                })
        except:
            pass
    if(len(tuitions) == 0):
        try:
            for k in seq(mapper.keys()).filter(lambda x: x in json_data['routing']['entry']):
                fee = json_data['routing']['entry']["furtherFeeInformation"]
                duration = json_data['routing']['entry']['duration']
                if not fee:
                    continue

                fee = re.findall(r'£\d+,\d+', fee).pop(0)

                student_category, study_mode = mapper[k]
                tuitions.append({
                    'study_mode': study_mode,
                    'duration': duration,
                    'student_category': student_category,
                    'fee': fee
                })
        except:
            pass
    if(len(tuitions) == 0):
        try:
            for k in seq(mapper.keys()).filter(lambda x: x in json_data['routing']['entry']):
                fee = json_data['routing']['entry']["onlineCourseFeeInformation"]
                duration = json_data['routing']['entry']['duration']
                if not fee:
                    continue

                fee = re.findall(r'&pound;(\d+,\d+)', fee).pop(0)

                student_category, study_mode = mapper[k]
                tuitions.append({
                    'study_mode': study_mode,
                    'duration': duration,
                    'student_category': student_category,
                    'fee': f'£{fee}'


                })
        except:
            pass
        
    
except IndexError:
    tuitions = []

return tuitions


[{'study_mode': 'Full-time', 'duration': 'MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years.', 'student_category': 'uk', 'fee': '£1,654'}, {'study_mode': 'Full-time', 'duration': 'MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years.', 'student_category': 'international', 'fee': '£1,654'}, {'study_mode': 'Part-time', 'duration': 'MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years.', 'student_category': 'uk', 'fee': '£1,654'}, {'study_mode': 'Part-Time', 'duration': 'MSc: Minimum two years, maximum six years. PG Dip: Minimum 16 months, maximum four years. PG Cert: Minimum eight months, maximum three years.', 'student_category': 'international', 'fee': '£1,654'}]


Playwright Section
---Use if needed---

In [22]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

await page.goto("https://www.sheffield.ac.uk/postgraduate/taught/courses/2024/advanced-clinical-practice-gp-mmedsci")
content = await page.content()

await browser.close()
await playwright.stop()
soup = BeautifulSoup(content, "html.parser")

In [11]:
text = "1 year, full-time\n2–3 years part-time\n 2/3 years"

# Define the regex pattern to match the previous word before "year" or "years"
pattern = r'(?<=\b)\S+\s+(?=year|years)'

# Search for the pattern in the text
matches = re.findall(pattern, text)
print(matches)

['1 ', '2–3 ', '2/3 ']
